# Q6a

In [1]:
import numpy as np
import pandas as pd
import random
import math
import random
df = pd.read_csv('hp_dataset.csv')
#df = df.drop(['stories', 'driveway','recroom','fullbase','gashw','airco','garagepl','prefarea'], axis=1)
df=df.loc[:,"price":"bathrms":1]
print(df)

        price  lotsize  bedrooms  bathrms
0     42000.0     5850         3        1
1     38500.0     4000         2        1
2     49500.0     3060         3        1
3     60500.0     6650         3        1
4     61000.0     6360         2        1
..        ...      ...       ...      ...
541   91500.0     4800         3        2
542   94000.0     6000         3        2
543  103000.0     6000         3        2
544  105000.0     6000         3        2
545  105000.0     6000         3        1

[546 rows x 4 columns]


In [2]:
# df_train=df.iloc[0:383]
df_test=df.iloc[383:546]
# df_test
df_train=df.iloc[0:383]
y = df_train.iloc[:,0]
x0 = [1]*len(y)
x1 = df_train.iloc[:,1]
x2 = df_train.iloc[:,2]
x3 = df_train.iloc[:,3]
x=list(zip(x0,x1,x2,x3))
y=np.matrix(y)
x=np.matrix(x)
y=y.transpose()
w=np.linalg.inv(x.transpose()*x)*x.transpose()*y
w0=w[0][0].item()
w1=w[1][0].item()
w2=w[2][0].item()
w3=w[3][0].item()
testprice=df_test.iloc[:,0]
area=df_test.iloc[:,1]
bed=df_test.iloc[:,2]
bath=df_test.iloc[:,3]
errorsum=0
for i in range(383,546):
    errorsum+=(abs(testprice[i]-(w0*1+w1*area[i]+w2*bed[i]+w3*bath[i])))/testprice[i]
errorperc=errorsum*100/163
print("Error Percentage without Regularisation = ",errorperc)

Error Percentage without Regularisation =  22.177301703033255


In [7]:
def ridge(lm):
    lam=lm
    df_test=df.iloc[383:546]
    df_train=df.iloc[0:383]
    y = df_train.iloc[:,0]
    x0 = [1]*len(y)
    x1 = df_train['lotsize'].tolist()
    x1=[((i-df_train['lotsize'].mean())/(df_train['lotsize'].max()-df_train['lotsize'].min())) for i in x1]
    x2 = df_train['bedrooms'].tolist()
    x2=[((i-df_train['bedrooms'].mean())/(df_train['bedrooms'].max()-df_train['bedrooms'].min())) for i in x2]
    x3 = df_train['bathrms'].tolist()
    x3=[((i-df_train['bathrms'].mean())/(df_train['bathrms'].max()-df_train['bathrms'].min())) for i in x3]
    x=list(zip(x0,x1,x2,x3))
    y=np.matrix(y)
    x=np.matrix(x)
    y=y.transpose()
    L=np.identity(4)
    L[0][0]=0;
    w=np.linalg.inv(x.transpose()*x+lam*L)*x.transpose()*y
    w0=w[0][0].item()
    w1=w[1][0].item()
    w2=w[2][0].item()
    w3=w[3][0].item()
    testprice=df_test['price'].tolist()
    area=df_test['lotsize'].tolist()
    area=[((i-df_test['lotsize'].mean())/(df_test['lotsize'].max()-df_test['lotsize'].min())) for i in area]
    bed=df_test['bedrooms'].tolist()
    bed=[((i-df_test['bedrooms'].mean())/(df_test['bedrooms'].max()-df_test['bedrooms'].min())) for i in bed]
    bath=df_test['bathrms'].tolist()
    bath=[((i-df_test['bathrms'].mean())/(df_test['bathrms'].max()-df_test['bathrms'].min())) for i in bath]
    errorsum=0
    for i in range(0,163):
        errorsum=errorsum+(abs(testprice[i]-(w0*1+w1*area[i]+w2*bed[i]+w3*bath[i])))/testprice[i]
    errorperc=errorsum*100/163
    return errorperc
min_error=1000000
min_lambda=0
for i in range(1,101):
    if ridge(i)<min_error:
        min_lambda=i
        min_error=ridge(i)
print("With Regularisation, lowest error percentage is ",min_error, "for Lambda =",min_lambda)

With Regularisation, lowest error percentage is  19.73341861715634 for Lambda = 40


# Q6b

In [81]:
def Slope(C, F, P, ind):
    Error = 0
    for i in range(len(F)):
        itr = 0
        for j in range(len(C)):
            itr = itr + C[j] * F[i][j]
        Error += (itr - P[i]) * F[i][ind]
    return Error
def scale(A):
    scaled=[]
    _mean=np.mean(A)
    _max=max(A)
    _min=min(A)
    for i in A:
        scaled.append((i - _mean) / (_max - _min))
    return scaled
def bgd():
    y = list(df_train.iloc[:,0])
    x0 = [1]*len(y)
    x1 = list(df_train.iloc[:,1])
    x2 = list(df_train.iloc[:,2])
    x3 = list(df_train.iloc[:,3])
    FeaturesTrain = []
    for i in range(383):
        FeaturesTrain.append([x0[i], x1[i], x2[i], x3[i]])
    LearningRate = 0.00000001
    Coeff = [0, 0, 0, 0]
    PriceTrain=[i for i in y]
    for i in range(5000):
        TempCoeff = Coeff.copy()
        for j in range(len(Coeff)):
            TempCoeff[j] = TempCoeff[j] - ((LearningRate / len(FeaturesTrain)) * (Slope(Coeff, FeaturesTrain, PriceTrain, j)))
        Coeff = TempCoeff.copy()
    y = list(df_test.iloc[:,0])
    x0 = [1]*len(y)
    x1 = list(df_test.iloc[:,1])
    x2 = list(df_test.iloc[:,2])
    x3 = list(df_test.iloc[:,3])
    PTest =[]
    FTest = []
    for i in range(0, 163):
        FTest.append([x0[i], x1[i], x2[i], x3[i]])
        PTest.append(y[i])
    Error = 0
    for i in range(len(FTest)):
        predicted = 0
        for j in range(len(Coeff)):
            predicted = predicted + Coeff[j] * FTest[i][j]
        Error += abs(predicted - PTest[i]) / PTest[i]
    Error = (Error / len(FTest)) * 100
    print("Mean absolute percentage error for Batch Gradient Descent without scaling is: " + str(Error))

def bgd_scaling():
    y = scale(list(df_train.iloc[:,0]))
    x0 = [1]*len(y)
    x1 = scale(list(df_train.iloc[:,1]))
    x2 = scale(list(df_train.iloc[:,2]))
    x3 = scale(list(df_train.iloc[:,3]))
    FeaturesTrain = []
    for i in range(383):
        FeaturesTrain.append([x0[i], x1[i], x2[i], x3[i]])
    LearningRate = 0.0001
    Coeff = [0, 0, 0, 0]
    PriceTrain=[i for i in y]
    for i in range(5000):
        TempCoeff = Coeff.copy()
        for j in range(len(Coeff)):
            TempCoeff[j] = TempCoeff[j] - ((LearningRate / len(FeaturesTrain)) * (Slope(Coeff, FeaturesTrain, PriceTrain, j)))
        Coeff = TempCoeff.copy()
    y = scale(list(df_test.iloc[:,0]))
    x0 = [1]*len(y)
    x1 = scale(list(df_test.iloc[:,1]))
    x2 = scale(list(df_test.iloc[:,2]))
    x3 = scale(list(df_test.iloc[:,3]))
    PTest =[]
    FTest = []
    for i in range(0, 163):
        FTest.append([x0[i], x1[i], x2[i], x3[i]])
        PTest.append(y[i])
    Error = 0
    for i in range(len(FTest)):
        predicted = 0
        for j in range(len(Coeff)):
            predicted = predicted + Coeff[j] * FTest[i][j]
        Error += abs(predicted - PTest[i]) / PTest[i]
    Error = (Error / len(FTest)) * 100
    print("Mean absolute percentage error for Batch Gradient Descent with scaling is: " + str(Error))
def Slopesgd(Coeff,FTrain,ActualVal,ind):
    itr = 0
    for j in range(len(Coeff)):
        itr = itr + Coeff[j]*FTrain[j]
    return (itr - ActualVal) * FTrain[ind]
def bgd():
    y = list(df_train.iloc[:,0])
    x0 = [1]*len(y)
    x1 = list(df_train.iloc[:,1])
    x2 = list(df_train.iloc[:,2])
    x3 = list(df_train.iloc[:,3])
    train_x = []
    for i in range(383):
        train_x.append([x0[i], x1[i], x2[i], x3[i]])
    Rate = 0.00000001
    Coeff = [0]*4
    train_y=[i for i in y]
    for i in range(5000):
        TempCoeff = Coeff.copy()
        for j in range(len(Coeff)):
            TempCoeff[j] = TempCoeff[j] - ((Rate / len(train_x)) * (Slope(Coeff, train_x, train_y, j)))
        Coeff = TempCoeff.copy()
    y = list(df_test.iloc[:,0])
    x0 = [1]*len(y)
    x1 = list(df_test.iloc[:,1])
    x2 = list(df_test.iloc[:,2])
    x3 = list(df_test.iloc[:,3])
    test_y =[]
    test_x = []
    for i in range(0, 163):
        test_x.append([x0[i], x1[i], x2[i], x3[i]])
        test_y.append(y[i])
    Error = 0
    for i in range(len(test_x)):
        predicted = 0
        for j in range(len(Coeff)):
            predicted = predicted + Coeff[j] * test_x[i][j]
        Error += abs(predicted - test_y[i]) / test_y[i]
    Error = (Error / len(test_x)) * 100
    print("Mean absolute percentage error for Batch Gradient Descent without scaling is: " ,Error)

def bgd_scaling():
    y = scale(list(df_train.iloc[:,0]))
    x0 = [1]*len(y)
    x1 = scale(list(df_train.iloc[:,1]))
    x2 = scale(list(df_train.iloc[:,2]))
    x3 = scale(list(df_train.iloc[:,3]))
    train_x = []
    for i in range(383):
        train_x.append([x0[i], x1[i], x2[i], x3[i]])
    Rate = 0.0001
    Coeff = [0]*4
    train_y=[i for i in y]
    for i in range(5000):
        TempCoeff = Coeff.copy()
        for j in range(len(Coeff)):
            TempCoeff[j] = TempCoeff[j] - ((Rate / len(train_x)) * (Slope(Coeff, train_x, train_y, j)))
        Coeff = TempCoeff.copy()
    y = scale(list(df_test.iloc[:,0]))
    x0 = [1]*len(y)
    x1 = scale(list(df_test.iloc[:,1]))
    x2 = scale(list(df_test.iloc[:,2]))
    x3 = scale(list(df_test.iloc[:,3]))
    test_y =[]
    test_x = []
    for i in range(0, 163):
        test_x.append([x0[i], x1[i], x2[i], x3[i]])
        test_y.append(y[i])
    Error = 0
    for i in range(len(test_x)):
        predicted = 0
        for j in range(len(Coeff)):
            predicted = predicted + Coeff[j] * test_x[i][j]
        Error += abs(predicted - test_y[i]) / test_y[i]
    Error = (Error / len(test_x)) * 100
    print("Mean absolute percentage error for Batch Gradient Descent with scaling is: " ,Error)
def Slopesgd(Coeff,FTrain,ActualVal,ind):
    itr = 0
    for j in range(len(Coeff)):
        itr = itr + Coeff[j]*FTrain[j]
    return (itr - ActualVal) * FTrain[ind]
def sgd():
    y = list(df_train.iloc[:,0])
    x0 = [1]*len(y)
    x1 = list(df_train.iloc[:,1])
    x2 = list(df_train.iloc[:,2])
    x3 = list(df_train.iloc[:,3])
    train_x = []
    for i in range(383):
        train_x.append([x0[i], x1[i], x2[i], x3[i]])
    train_y=y
    RateNoScaling = 0.0000000005
    Coeff = [0]*4
    for iter in range(100):
        for i in range(len(train_y)):
            TempCoeff = Coeff.copy()
            for j in range(4):
                TempCoeff[j] = TempCoeff[j] - (RateNoScaling * (Slopesgd(Coeff, train_x[i], train_y[i], j)))
            Coeff = TempCoeff.copy()

    y = list(df_test.iloc[:,0])
    x0 = [1]*len(y)
    x1 = list(df_test.iloc[:,1])
    x2 = list(df_test.iloc[:,2])
    x3 = list(df_test.iloc[:,3])
    test_y =[]
    test_x = []
    for i in range(0, 163):
        test_x.append([x0[i], x1[i], x2[i], x3[i]])
        test_y.append(y[i])
    Error = 0
    for i in range(len(test_x)):
        predicted = 0
        for j in range(len(Coeff)):
            predicted = predicted + Coeff[j] * test_x[i][j]
        Error += abs(predicted - test_y[i]) / test_y[i]
    Error = (Error / len(test_x)) * 100
    print("Mean absolute percentage error for Stochastic Gradient Descent without scaling is: " ,Error)
def sgd_scaling():
    y = scale(list(df_train.iloc[:,0]))
    x0 = [1]*len(y)
    x1 = scale(list(df_train.iloc[:,1]))
    x2 = scale(list(df_train.iloc[:,2]))
    x3 = scale(list(df_train.iloc[:,3]))
    train_x = []
    for i in range(383):
        train_x.append([x0[i], x1[i], x2[i], x3[i]])
    train_y=y
    Rate = 0.003
    Coeff = [0]*4
    for iter in range(10):
        for i in range(len(train_y)):
            TempCoeff = Coeff.copy()
            for j in range(4):
                TempCoeff[j] = TempCoeff[j] - (Rate * (Slopesgd(Coeff, train_x[i], train_y[i], j)))
            Coeff = TempCoeff.copy()
    y = list(df_test.iloc[:,0])
    x0 = [1]*len(y)
    x1 = list(df_test.iloc[:,1])
    x2 = list(df_test.iloc[:,2])
    x3 = list(df_test.iloc[:,3])
    test_y =[]
    test_x = []
    for i in range(0, 163):
        test_x.append([x0[i], x1[i], x2[i], x3[i]])
        test_y.append(y[i])
    Error = 0
    for i in range(len(test_x)):
        predicted = 0
        for j in range(len(Coeff)):
            predicted = predicted + Coeff[j] * test_x[i][j]
        Error += abs(predicted - test_y[i]) / test_y[i]
    Error = (Error / len(test_x)) * 100
    print("Mean absolute percentage error for Stochastic Gradient Descent with scaling is: " ,Error)

def mbgd():
    y = scale(list(df_train.iloc[:,0]))
    x0 = [1]*len(y)
    x1 = scale(list(df_train.iloc[:,1]))
    x2 = scale(list(df_train.iloc[:,2]))
    x3 = scale(list(df_train.iloc[:,3]))
    train_x = []
    for i in range(383):
        train_x.append([x0[i], x1[i], x2[i], x3[i]])
    train_y=y
    BatchSize = 30;
    Rate = 0.000000001
    Coeff = [0]*4
    NoOfBatches = math.ceil(len(train_y) / BatchSize)
    equallyDiv = False
    if (len(train_y) % BatchSize == 0):
        equallyDiv = True;

    for epoch in range(30):
        for batch in range(NoOfBatches):
            Summation = [0]*4
            for j in range(len(Coeff)):
                for i in range(BatchSize):
                    if (batch * BatchSize + i == len(train_x)):
                        break
                    PValue = 0.0
                    for wj in range(len(Coeff)):
                        PValue += Coeff[wj] * train_x[batch * BatchSize + i][wj]
                    PValue -= train_y[batch * BatchSize + i]
                    PValue *= train_x[batch * BatchSize + i][j]
                    Summation[j] += PValue;

            if (not equallyDiv and batch == NoOfBatches - 1):
                for j in range(len(Summation)):
                    Coeff[j] -= (Summation[j] / (len(train_y) % BatchSize)) * Rate
            else:
                for j in range(len(Summation)):
                    Coeff[j] -= (Summation[j] / BatchSize) * Rate

    y = list(df_test.iloc[:,0])
    x0 = [1]*len(y)
    x1 = list(df_test.iloc[:,1])
    x2 = list(df_test.iloc[:,2])
    x3 = list(df_test.iloc[:,3])
    test_y =[]
    test_x = []
    for i in range(0, 163):
        test_x.append([x0[i], x1[i], x2[i], x3[i]])
        test_y.append(y[i])
    Error = 0
    for i in range(len(test_x)):
        predicted = 0
        for j in range(len(Coeff)):
            predicted = predicted + Coeff[j] * test_x[i][j]
        Error += abs(predicted - test_y[i]) / test_y[i]
    Error = (Error / len(test_x)) * 100
    print("Mean absolute percentage error for Mini Batch Gradient Descent without scaling is: " ,Error)
def mbgd_scaling():
    y = scale(list(df_train.iloc[:,0]))
    x0 = [1]*len(y)
    x1 = scale(list(df_train.iloc[:,1]))
    x2 = scale(list(df_train.iloc[:,2]))
    x3 = scale(list(df_train.iloc[:,3]))
    train_x = []
    for i in range(383):
        train_x.append([x0[i], x1[i], x2[i], x3[i]])
    train_y=y

    BatchSize = 30;
    Rate = 0.0025
    Coeff = [0]*4
    NoOfBatches = math.ceil(len(train_y) / BatchSize)
    equallyDiv = False
    if (len(train_y) % BatchSize == 0):
        equallyDiv = True;

    for epoch in range(30):
        for batch in range(NoOfBatches):
            Summation = [0]*4
            for j in range(len(Coeff)):
                for i in range(BatchSize):
                    if (batch * BatchSize + i == len(train_x)):
                        break
                    PValue = 0.0
                    for wj in range(len(Coeff)):
                        PValue += Coeff[wj] * train_x[batch * BatchSize + i][wj]
                    PValue -= train_y[batch * BatchSize + i]
                    PValue *= train_x[batch * BatchSize + i][j]
                    Summation[j] += PValue;

            if (not equallyDiv and batch == NoOfBatches - 1):
                for j in range(len(Summation)):
                    Coeff[j] -= (Summation[j] / (len(train_y) % BatchSize)) * Rate
            else:
                for j in range(len(Summation)):
                    Coeff[j] -= (Summation[j] / BatchSize) * Rate

    y = list(df_test.iloc[:,0])
    x0 = [1]*len(y)
    x1 = list(df_test.iloc[:,1])
    x2 = list(df_test.iloc[:,2])
    x3 = list(df_test.iloc[:,3])
    test_y =[]
    test_x = []
    for i in range(0, 163):
        test_x.append([x0[i], x1[i], x2[i], x3[i]])
        test_y.append(y[i])
    Error = 0
    for i in range(len(test_x)):
        predicted = 0
        for j in range(len(Coeff)):
            predicted = predicted + Coeff[j] * test_x[i][j]
        Error += abs(predicted - test_y[i]) / test_y[i]
    Error = (Error / len(test_x)) * 100
    print("Mean absolute percentage error for Mini Batch Gradient Descent with scaling is : ",Error)
bgd()
bgd_scaling()
sgd()
sgd_scaling()
mbgd()
mbgd_scaling()

Mean absolute percentage error for Batch Gradient Descent without scaling is:  29.976669944053334
Mean absolute percentage error for Batch Gradient Descent with scaling is:  6.650598257264065
Mean absolute percentage error for Stochastic Gradient Descent without scaling is:  29.95951437900304
Mean absolute percentage error for Stochastic Gradient Descent with scaling is:  98.93654572968711
Mean absolute percentage error for Mini Batch Gradient Descent without scaling is:  99.99999995347812
Mean absolute percentage error for Mini Batch Gradient Descent with scaling is :  99.88543493572118


In [55]:
def Slopebgd(C, F, P, ind):
    Error = 0
    for i in range(len(F)):
        h = 0
        for j in range(len(C)):
            h = h + C[j] * F[i][j]
        Error += (h - P[i]) * F[i][ind]
    return Error
def Slopembgd(C, F, P, ind,bsize,k):
    Error = 0
    for i in range(k,k+bsize):
        h = 0
        for j in range(len(C)):
            h = h + C[j] * F[i][j]
        Error += (h - P[i]) * F[i][ind]
    return Error
def Slopesgd(C,F,P,ind,i):
    h = 0
    for j in range(len(C)):
        h = h + C[j] * F[i][j]
    return (h - P[i]) * F[i][ind]
def scale(A):
    _mean=np.mean(A)
    _max=max(A)
    _min=min(A)
    scaled=[((i - _mean) / (_max - _min)) for i in A]
    return scaled
y = list(df_train.iloc[:,0])
x0 = [1]*len(y)
x1 = list(df_train.iloc[:,1])
x2 = list(df_train.iloc[:,2])
x3 = list(df_train.iloc[:,3])
FTrain=list(zip(x0,x1,x2,x3))
PTrain=y
y = list(df_test.iloc[:,0])
x0 = [1]*len(y)
x1 = list(df_test.iloc[:,1])
x2 = list(df_test.iloc[:,2])
x3 = list(df_test.iloc[:,3])
PTest =y
FTest=list(zip(x0,x1,x2,x3))
y = list(df_train.iloc[:,0])
x0 = [1]*len(y)
x1 = scale(list(df_train.iloc[:,1]))
x2 = scale(list(df_train.iloc[:,2]))
x3 = scale(list(df_train.iloc[:,3]))
scaled_FTrain=list(zip(x0,x1,x2,x3))
scaled_PTrain=y
y = list(df_test.iloc[:,0])
x0 = [1]*len(y)
x1 = scale(list(df_test.iloc[:,1]))
x2 = scale(list(df_test.iloc[:,2]))
x3 = scale(list(df_test.iloc[:,3]))
scaled_PTest =y
scaled_FTest=list(zip(x0,x1,x2,x3))

def bgd(F1,P1,F2,P2,rate):
    LearningRate = rate
    Coeff = [0, 0, 0, 0]
    for i in range(50000):
        TempCoeff = Coeff.copy()
        for j in range(len(Coeff)):
            TempCoeff[j] = TempCoeff[j] - ((LearningRate / len(F1)) * (Slopebgd(Coeff, F1, P1, j)))
        Coeff = TempCoeff.copy()
    Error = 0
    for i in range(len(FTest)):
        predicted = 0
        for j in range(len(Coeff)):
            predicted = predicted + Coeff[j] * F2[i][j]
        Error += (abs(predicted - P2[i])/P2[i])
    Error = (Error / len(F2))*100
    return Error
# print(bgd(FTrain,PTrain,FTest,PTest,0.0001))
# print(bgd(scaled_FTrain,scaled_PTrain,scaled_FTest,scaled_PTest,0.001))
def sgd(F1,P1,F2,P2,rate):
    LearningRate = rate
    Coeff = [0,0,0,0]
    for iter in range(1000000):
        i=random.randint(0,382)
        TempCoeff = Coeff.copy()
        for j in range(4):
            TempCoeff[j] = TempCoeff[j] - ((LearningRate/len(F1)) * (Slopesgd(Coeff, F1, P1, j,i)))
        Coeff = TempCoeff.copy()

    Error = 0
    for i in range(len(F2)):
        predicted = 0
        for j in range(len(Coeff)):
            predicted = predicted + Coeff[j] * F2[i][j]
        Error += abs(predicted - P2[i]) / P2[i]
    Error = (Error / len(F2)) * 100
    return Error
# print(sgd(FTrain,PTrain,FTest,PTest,0.0000000005))
# print(sgd(scaled_FTrain,scaled_PTrain,scaled_FTest,scaled_PTest,0.003))
def mbgd(F1,P1,F2,P2,rate,bsize,epochs):
    LearningRate = rate
    Coeff = [0,0,0,0]
    NoOfBatches = math.floor(len(P1) / bsize)
    for epoch in range(epochs):
        batch = random.randrange(0,NoOfBatches)
        TempCoeff = Coeff.copy()
        for j in range(len(Coeff)):
            TempCoeff[j] = TempCoeff[j] - ((LearningRate / len(F1)) * (Slopembgd(Coeff, F1, P1, j,bsize,bsize*batch)))
        Coeff = TempCoeff.copy()
        
    Error = 0
    for i in range(len(F2)):
        predicted = 0
        for j in range(len(Coeff)):
            predicted = predicted + Coeff[j] * F2[i][j]
        Error += abs(predicted - P2[i]) / P2[i]
    Error = (Error / len(F2)) * 100
    return Error
# print(mbgd(FTrain,PTrain,FTest,PTest,0.000000001,10,100000))
print(mbgd(scaled_FTrain,scaled_PTrain,scaled_FTest,scaled_PTest,0.0025,10,100000))

20.089282852017828


# Without Regularization

In [ ]:
def Slopebgd(C, F, P, ind):
    Error = 0
    for i in range(len(F)):
        h = 0
        for j in range(len(C)):
            h = h + C[j] * F[i][j]
        Error += (h - P[i]) * F[i][ind]
    return Error
def Slopembgd(C, F, P, ind,bsize,k):
    Error = 0
    for i in range(k,k+bsize):
        h = 0
        for j in range(len(C)):
            h = h + C[j] * F[i][j]
        Error += (h - P[i]) * F[i][ind]
    return Error
def Slopesgd(C,F,P,ind,i):
    h = 0
    for j in range(len(C)):
        h = h + C[j] * F[i][j]
    return (h - P[i]) * F[i][ind]
def scale(A):
    _mean=np.mean(A)
    _max=max(A)
    _min=min(A)
    scaled=[((i - _mean) / (_max - _min)) for i in A]
    return scaled
y = list(df_train.iloc[:,0])
x0 = [1]*len(y)
x1 = scale(list(df_train.iloc[:,1]))
x2 = scale(list(df_train.iloc[:,2]))
x3 = scale(list(df_train.iloc[:,3]))
scaled_FTrain=list(zip(x0,x1,x2,x3))
scaled_PTrain=y
y = list(df_test.iloc[:,0])
x0 = [1]*len(y)
x1 = scale(list(df_test.iloc[:,1]))
x2 = scale(list(df_test.iloc[:,2]))
x3 = scale(list(df_test.iloc[:,3]))
scaled_PTest =y
scaled_FTest=list(zip(x0,x1,x2,x3))

def bgd(F1,P1,F2,P2,rate):
    LearningRate = rate
    Coeff = [0, 0, 0, 0]
    for i in range(50000):
        TempCoeff = Coeff.copy()
        for j in range(len(Coeff)):
            TempCoeff[j] = TempCoeff[j] - ((LearningRate / len(F1)) * (Slopebgd(Coeff, F1, P1, j)))
        Coeff = TempCoeff.copy()
    Error = 0
    for i in range(len(FTest)):
        predicted = 0
        for j in range(len(Coeff)):
            predicted = predicted + Coeff[j] * F2[i][j]
        Error += (abs(predicted - P2[i])/P2[i])
    Error = (Error / len(F2))*100
    return Error
print(bgd(scaled_FTrain,scaled_PTrain,scaled_FTest,scaled_PTest,0.001))
def sgd(F1,P1,F2,P2,rate):
    LearningRate = rate
    Coeff = [0,0,0,0]
    for iter in range(1000000):
        i=random.randint(0,382)
        TempCoeff = Coeff.copy()
        for j in range(4):
            TempCoeff[j] = TempCoeff[j] - ((LearningRate/len(F1)) * (Slopesgd(Coeff, F1, P1, j,i)))
        Coeff = TempCoeff.copy()

    Error = 0
    for i in range(len(F2)):
        predicted = 0
        for j in range(len(Coeff)):
            predicted = predicted + Coeff[j] * F2[i][j]
        Error += abs(predicted - P2[i]) / P2[i]
    Error = (Error / len(F2)) * 100
    return Error
print(sgd(scaled_FTrain,scaled_PTrain,scaled_FTest,scaled_PTest,0.003))
def mbgd(F1,P1,F2,P2,rate,bsize,epochs):
    LearningRate = rate
    Coeff = [0,0,0,0]
    NoOfBatches = math.floor(len(P1) / bsize)
    for epoch in range(epochs):
        batch = random.randrange(0,NoOfBatches)
        TempCoeff = Coeff.copy()
        for j in range(len(Coeff)):
            TempCoeff[j] = TempCoeff[j] - ((LearningRate / len(F1)) * (Slopembgd(Coeff, F1, P1, j,bsize,bsize*batch)))
        Coeff = TempCoeff.copy()
        
    Error = 0
    for i in range(len(F2)):
        predicted = 0
        for j in range(len(Coeff)):
            predicted = predicted + Coeff[j] * F2[i][j]
        Error += abs(predicted - P2[i]) / P2[i]
    Error = (Error / len(F2)) * 100
    return Error
print(mbgd(scaled_FTrain,scaled_PTrain,scaled_FTest,scaled_PTest,0.0025,10,100000))

# With Regularization

In [56]:
def Slopebgd(C, F, P, ind):
    Error = 0
    for i in range(len(F)):
        h = 0
        for j in range(len(C)):
            h = h + C[j] * F[i][j]
        Error += (h - P[i]) * F[i][ind]
    return Error
def Slopembgd(C, F, P, ind,bsize,k):
    Error = 0
    for i in range(k,k+bsize):
        h = 0
        for j in range(len(C)):
            h = h + C[j] * F[i][j]
        Error += (h - P[i]) * F[i][ind]
    return Error
def Slopesgd(C,F,P,ind,i):
    h = 0
    for j in range(len(C)):
        h = h + C[j] * F[i][j]
    return (h - P[i]) * F[i][ind]
def scale(A):
    _mean=np.mean(A)
    _max=max(A)
    _min=min(A)
    scaled=[((i - _mean) / (_max - _min)) for i in A]
    return scaled
y = list(df_train.iloc[:,0])
x0 = [1]*len(y)
x1 = scale(list(df_train.iloc[:,1]))
x2 = scale(list(df_train.iloc[:,2]))
x3 = scale(list(df_train.iloc[:,3]))
scaled_FTrain=list(zip(x0,x1,x2,x3))
scaled_PTrain=y
y = list(df_test.iloc[:,0])
x0 = [1]*len(y)
x1 = scale(list(df_test.iloc[:,1]))
x2 = scale(list(df_test.iloc[:,2]))
x3 = scale(list(df_test.iloc[:,3]))
scaled_PTest =y
scaled_FTest=list(zip(x0,x1,x2,x3))

def bgd(F1,P1,F2,P2,rate,lam):
    LearningRate = rate
    Coeff = [0, 0, 0, 0]
    for i in range(50000):
        TempCoeff = Coeff.copy()
        for j in range(len(Coeff)):
            TempCoeff[j] = TempCoeff[j] - ((LearningRate / len(F1)) * (Slopebgd(Coeff, F1, P1, j)+lam*TempCoeff[j]))
        Coeff = TempCoeff.copy()
    Error = 0
    for i in range(len(FTest)):
        predicted = 0
        for j in range(len(Coeff)):
            predicted = predicted + Coeff[j] * F2[i][j]
        Error += (abs(predicted - P2[i])/P2[i])
    Error = (Error / len(F2))*100
    return Error
print(bgd(scaled_FTrain,scaled_PTrain,scaled_FTest,scaled_PTest,0.001,1))
def sgd(F1,P1,F2,P2,rate,lam):
    LearningRate = rate
    Coeff = [0,0,0,0]
    for iter in range(1000000):
        i=random.randint(0,382)
        TempCoeff = Coeff.copy()
        for j in range(4):
            TempCoeff[j] = TempCoeff[j] - ((LearningRate/len(F1)) * (Slopesgd(Coeff, F1, P1, j,i)+lam*TempCoeff[j]))
        Coeff = TempCoeff.copy()

    Error = 0
    for i in range(len(F2)):
        predicted = 0
        for j in range(len(Coeff)):
            predicted = predicted + Coeff[j] * F2[i][j]
        Error += abs(predicted - P2[i]) / P2[i]
    Error = (Error / len(F2)) * 100
    return Error
print(sgd(scaled_FTrain,scaled_PTrain,scaled_FTest,scaled_PTest,0.003,1))
def mbgd(F1,P1,F2,P2,rate,bsize,epochs,lam):
    LearningRate = rate
    Coeff = [0,0,0,0]
    NoOfBatches = math.floor(len(P1) / bsize)
    for epoch in range(epochs):
        batch = random.randrange(0,NoOfBatches)
        TempCoeff = Coeff.copy()
        for j in range(len(Coeff)):
            TempCoeff[j] = TempCoeff[j] - ((LearningRate / len(F1)) * (Slopembgd(Coeff, F1, P1, j,bsize,bsize*batch)+lam*TempCoeff[j]))
        Coeff = TempCoeff.copy()
        
    Error = 0
    for i in range(len(F2)):
        predicted = 0
        for j in range(len(Coeff)):
            predicted = predicted + Coeff[j] * F2[i][j]
        Error += abs(predicted - P2[i]) / P2[i]
    Error = (Error / len(F2)) * 100
    return Error
print(mbgd(scaled_FTrain,scaled_PTrain,scaled_FTest,scaled_PTest,0.0025,10,100000,1))

24.474098485461486
